In [7]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
import shutil

from hloc import (
    extract_features,
    match_features,
    match_dense,
    reconstruction,
    visualization,
    pairs_from_all,
    localize_sfm,
    pairs_from_exhaustive
)

ROOT = Path().absolute()
# add root to path in order to execute scripts
if str(ROOT) not in sys.path:
  sys.path.append(str(ROOT))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
images = Path("datasets/framlingham/images")

outputs = Path("outputs/sfm")
sfm_pairs = outputs / "pairs.txt"
sfm_dir = outputs / "test_dbs"
sfm_images = sfm_dir / "images"

# retrieval_conf = extract_features.confs["netvlad"]
feature_conf = extract_features.confs["superpoint_max"]
matcher_conf = match_features.confs["superpoint+lightglue"]
# matcher_conf = match_dense.confs["loftr"]

In [9]:
pairs_from_all.main(sfm_pairs, images)

[2024/06/18 15:47:03 hloc.utils.parsers INFO] Imported 11 images from images
[2024/06/18 15:47:03 hloc INFO] Found 55 pairs.


In [10]:
feature_path = extract_features.main(feature_conf, images, outputs)
#feature_path = Path(outputs, feature_conf["output"] + ".h5")

[2024/06/18 15:47:03 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-rmax1600',
 'preprocessing': {'grayscale': True, 'resize_force': True, 'resize_max': 1600}}
[2024/06/18 15:47:03 hloc INFO] Found 11 images in root datasets/framlingham/images.


Loaded SuperPoint model


  0%|          | 0/11 [00:00<?, ?it/s]

dict_keys(['image'])


 18%|█▊        | 2/11 [00:00<00:02,  3.70it/s]

...

 82%|████████▏ | 9/11 [00:01<00:00,  4.99it/s]

dict_keys(['image'])


 91%|█████████ | 10/11 [00:02<00:00,  4.85it/s]

dict_keys(['image'])
dict_keys(['image'])


100%|██████████| 11/11 [00:02<00:00,  4.79it/s]
[2024/06/18 15:47:06 hloc INFO] Finished exporting features.


In [11]:
match_path = match_features.main(
    conf=matcher_conf,
    pairs=sfm_pairs,
    features=feature_conf["output"],
    export_dir=outputs,
)

"""
feature_path, match_path = match_dense.main(
  conf=matcher_conf, 
  pairs=sfm_pairs, 
  image_dir=images, 
  export_dir=outputs,
  features_ref=feature_path
)
"""

[2024/06/18 15:47:06 hloc INFO] Matching local features with configuration:
{'model': {'features': 'superpoint', 'name': 'lightglue'},
 'output': 'matches-superpoint-lightglue'}
  0%|          | 0/55 [00:00<?, ?it/s]

dict_keys(['descriptors0', 'image_size0', 'keypoints0', 'scores0', 'image0', 'descriptors1', 'image_size1', 'keypoints1', 'scores1', 'image1'])


  2%|▏         | 1/55 [00:00<00:18,  2.95it/s]

dict_keys(['descriptors0', 'image_size0', 'keypoints0', 'scores0', 'image0', 'descriptors1', 'image_size1', 'keypoints1', 'scores1', 'image1'])


  4%|▎         | 2/55 [00:00<00:18,  2.83it/s]

...

dict_keys(['descriptors0', 'image_size0', 'keypoints0', 'scores0', 'image0', 'descriptors1', 'image_size1', 'keypoints1', 'scores1', 'image1'])


 98%|█████████▊| 54/55 [00:17<00:00,  3.27it/s]

dict_keys(['descriptors0', 'image_size0', 'keypoints0', 'scores0', 'image0', 'descriptors1', 'image_size1', 'keypoints1', 'scores1', 'image1'])


100%|██████████| 55/55 [00:18<00:00,  3.01it/s]
[2024/06/18 15:47:24 hloc INFO] Finished exporting matches.


'\nfeature_path, match_path = match_dense.main(\n  conf=matcher_conf, \n  pairs=sfm_pairs, \n  image_dir=images, \n  export_dir=outputs,\n  features_ref=feature_path\n)\n'

## 3D reconstruction
Run COLMAP on the features and matches.

In [17]:
import pycolmap

# set focal length to 25mm for all images that are not query images
POST_IMAGE_IMPORT_SQL = """UPDATE cameras
SET prior_focal_length = 26
WHERE camera_id IN (
    SELECT c.camera_id
    FROM cameras c
    JOIN images i ON c.camera_id = i.camera_id
    WHERE i.name NOT LIKE '%MAIN%'
)"""

model = reconstruction.main(
    sfm_dir,
    images,
    sfm_pairs,
    feature_path,
    match_path,
    #camera_mode=pycolmap.CameraMode.PER_FOLDER,
    post_image_import_sql=POST_IMAGE_IMPORT_SQL,
)

# CameraMode.PER_FOLDER allows for different camera model per image folder

[2024/06/18 15:54:55 hloc WARNING] The database already exists, deleting it.
[2024/06/18 15:54:55 hloc INFO] Creating an empty database...
[2024/06/18 15:54:56 hloc INFO] Importing images into the database...
[2024/06/18 15:54:58 hloc INFO] Importing features into the database...


[2419.2  756.  1008.     0. ] 1
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 2
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 3
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 4
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 5
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 6
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 7
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 8
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 9
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]
[2419.2  756.  1008.     0. ] 10
[  26.  756. 1008.    0.]
[  26.  756. 1008.    0.]


100%|██████████| 11/11 [00:00<00:00, 136.79it/s]
[2024/06/18 15:54:58 hloc INFO] Importing matches into the database...
100%|██████████| 55/55 [00:00<00:00, 173.63it/s]
[2024/06/18 15:54:59 hloc INFO] Performing geometric verification of the matches...
I20240618 15:54:59.376794 16938 misc.cc:198] 
Custom feature matching
I20240618 15:54:59.381894 16938 feature_matching.cc:1021] Matching block [1/1]
I20240618 15:55:01.561766 16938 feature_matching.cc:46]  in 2.180s
I20240618 15:55:01.587783 16938 timer.cc:91] Elapsed time: 0.037 [minutes]
[2024/06/18 15:55:01 hloc INFO] Running 3D reconstruction...
I20240618 15:55:01.666093 16955 misc.cc:198] 
Loading database
I20240618 15:55:01.694367 16955 database_cache.cc:54] Loading cameras...
I20240618 15:55:01.695399 16955 database_cache.cc:64]  11 in 0.001s
I20240618 15:55:01.695446 16955 database_cache.cc:72] Loading matches...
I20240618 15:55:01.702869 16955 database_cache.cc:78]  55 in 0.007s
I20240618 15:55:01.702926 16955 database_cache.cc:

In [18]:
from third_party.Neuralangelo.convert_data_to_json import data_to_json

args = {
  "data_dir": str(sfm_dir.absolute()),
  "scene_type": "outdoor",
  "image_dir": str(images.absolute()),
}

data_to_json(args)

if not sfm_images.exists():
  # copy files from input images to self contained project path
  # os.mkdir(new_input_path)
  print("copied input images to self contained SfM reconstruction folder")
  shutil.copytree(images, sfm_images)

Fraction of images looking at the center: 0.00.
Fraction of images positioned around the center: 1.00.
Valid fraction of concentric images: 0.00.
frame reprojection error for image ID 3: 1.132
frame reprojection error for image ID 10: 1.132
Writing data to json file: /mnt/d/dev/python/Hierarchical-Localization/outputs/sfm/test_dbs/transforms.json
copied input images to self contained SfM reconstruction folder


## Visualization
We visualize some of the registered images, and color their keypoint by visibility, track length, or triangulated depth.

In [19]:
#visualization.visualize_sfm_2d(model, images, color_by="visibility", n=5)

In [20]:
#visualization.visualize_sfm_2d(model, images, color_by="track_length", n=5)

In [21]:
#visualization.visualize_sfm_2d(model, images, color_by="depth", n=5)